# Sweeps - Eigenmode matrix

### Prerequisite
You need to have a working local installation of Ansys

## 1. Perform the necessary imports and create a QDesign in Metal first.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
from qiskit_metal.analyses.quantization import EPRanalysis

In [ ]:
# Create the design in Metal
# Create a design by specifying the chip size and open Metal GUI.

design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'

gui = MetalGUI(design)

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

### In this example, the design consists of 1 qubit and 1 CPW connected to OpenToGround.

In [ ]:
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

# Remove all qcomponents from GUI.
design.delete_all_components()

# So as to demonstrate the quality factor outputs easily, the
#subtrate material type is being changed to FR4_epoxy from the
#default of silicon
design.chips.main.material = 'FR4_epoxy'

q1 = TransmonPocket(
    design,
    'Q1',
    options=dict(pad_width='425 um',
                 pocket_height='650um', 
                 hfss_inductance = '17nH',
                 connection_pads=dict(
                     readout=dict(loc_W=+1, loc_H=+1, pad_width='200um'))))
otg = OpenToGround(design,
                   'open_to_ground',
                   options=dict(pos_x='1.75mm', pos_y='0um', orientation='0'))
readout = RouteMeander(
    design, 'readout',
    Dict(
        total_length='6 mm',
        hfss_wire_bonds = True,
        fillet='90 um',
        lead=dict(start_straight='100um'),
        pin_inputs=Dict(start_pin=Dict(component='Q1', pin='readout'),
                        end_pin=Dict(component='open_to_ground', pin='open')),
    ))

gui.rebuild()
gui.autoscale()

## 2 Metal passes information to 'hfss' simulator, and gets a solution matrix.


In [ ]:
# Create a separate analysis object for the combined qbit+readout.
eig_qres = EPRanalysis(design, "hfss")


Prepare data to pass as arguments for method run_sweep().  

Method run_sweep() will open the simulation software if software is not open already.

In [ ]:
### for render_design()
# Render every QComponent in QDesign.
render_qcomps = []

# Identify which kind of pins in Ansys. 
# Follow details from renderer in
# QHFSSRenderer.render_design.
# No pins are open, so don't need to utilize render_endcaps.
open_terminations = []

#List of tuples of jj's that shouldn't be rendered.  
#Follow details from renderer in QHFSSRenderer.render_design.
render_ignored_jjs = []

# Either calculate a bounding box based on the location of 
# rendered geometries or use chip size from design class.
box_plus_buffer = True

In [ ]:
# For simulator hfss, the setup options are :
# min_freq_ghz, n_modes, max_delta_f, max_passes, min_passes, min_converged=None,
# pct_refinement, basis_order

# If you don't pass all the arguments, the default is determined by 
# QHFSSRenderer's default_options.

# If a setup named "sweeper_em_setup" exists in the project, it will be deleted, 
# and a new setup will be added.

eig_qres.sim.setup.name="sweeper_em_setup"
eig_qres.sim.setup.min_freq_ghz=4
eig_qres.sim.setup.n_modes=2
eig_qres.sim.setup.max_passes=15
eig_qres.sim.setup.min_converged = 2
eig_qres.sim.setup.max_delta_f = 0.2

eig_qres.setup.junctions.jj.rect = 'JJ_rect_Lj_Q1_rect_jj'
eig_qres.setup.junctions.jj.line = 'JJ_Lj_Q1_rect_jj_'


### - Connect to Ansys HFSS, eigenmode solution.
### - Rebuild QComponents in Metal.
### - Render QComponents within HFSS and setup.
### - Delete/Clear the HFSS between each calculation of solution matrix.
### - Calculate solution matrix for each value in option_sweep.

#### Return a dict and return code.  If the return code is zero, there were no errors detected.  
#### The dict has:  key = each value used to sweep, value = data from simulators

#### This could take minutes based size of design.

In [ ]:
#Note: The method will connect to Ansys, activate_eigenmode_design(), add_eigenmode_setup().

all_sweeps, return_code = eig_qres.run_sweep(readout.name,
                                        'total_length', 
                                        ['10mm', '11mm', '12mm'],
                                        render_qcomps,
                                        open_terminations,
                                         ignored_jjs=render_ignored_jjs,
                                        design_name="GetEigenModeSolution", 
                                       box_plus_buffer=box_plus_buffer
                                      )


In [ ]:
all_sweeps.keys()

In [ ]:
# For example, just one group of solution data.
all_sweeps['10mm'].keys()


In [ ]:
all_sweeps['10mm']


In [ ]:
all_sweeps['10mm']['variables']

In [ ]:
all_sweeps['10mm']['sim_variables']['convergence_t']

In [ ]:
all_sweeps['10mm']['sim_variables']['convergence_f']

In [ ]:
# Uncomment the next close simulation software. 
#eig_qres.sim.close()

In [ ]:
# Uncomment next line if you would like to close the gui
#gui.main_window.close()